# Scouting System Test

1. Ensure the PostgreSQL database is running. (Type `pg_ctl start` at a command prompt or start the PostgreSQL service using the Windows *Services* tool.

2. Select *Run All* from the Jupyter Notebook *Cell* menu.

### 1. Package Imports

In [9]:
# Import packages used by scouting system
import sys
import os
import re

import pandas as pd
import sqlalchemy as sa

In [10]:
# Import scouting system packages
import server.scouting.output as out
from server.scouting.event import EventDal

ModuleNotFoundError: No module named 'server'

### 2. Database Connection

In [ ]:
# Get a database connection
connection_string = ('postgresql://irs1318:irs1318'
                     '@localhost:5432/scouting')
engine = sa.create_engine(connection_string)
conn = engine.connect()

### 3. Plot Settings

In [ ]:
# This section is necessary for viewing plots in the notebook.
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
matplotlib.style.use("ggplot")

### 4. Set Event

In [ ]:
# Set Current Event
current_event = "turing"
EventDal.set_current_event(current_event)

### 5. Get a Dataframe Containing all Measures
Verify a Pandas dataframe (table) is visible below this cell. 

In [ ]:
# Get Measures Dataframe
sql_measures = sa.text(
    "SELECT events.name AS event, matches.name AS match, levels.name AS level, dates.name AS date, "
    "teams.name AS team, alliances.name AS alliance, "
    "stations.name AS station, phases.name AS phase, actors.name AS actor, tasks.name AS task, "
    "measures.capability AS capability, measures.successes as successes, measures.attempts AS attempts, "
    "measures.cycle_times as cycle_times, measuretypes.name AS measuretype, reasons.name AS reason, "
    "attempts.name AS attempt "
    "FROM ((((((((((((measures FULL OUTER JOIN teams ON measures.team_id=teams.id) "
    "LEFT JOIN events ON measures.event_id=events.id) "
    "LEFT JOIN matches ON measures.match_id=matches.id) "
    "LEFT JOIN levels ON measures.level_id=levels.id) "
    "LEFT JOIN dates ON measures.date_id=dates.id) "
    "LEFT JOIN alliances ON measures.alliance_id=alliances.id) "
    "LEFT JOIN stations ON measures.station_id=stations.id) "
    "LEFT JOIN phases ON measures.phase_id=phases.id) "
    "LEFT JOIN actors ON measures.actor_id=actors.id) "
    "LEFT JOIN tasks ON measures.task_id=tasks.id) "
    "LEFT JOIN reasons ON measures.reason_id=reasons.id) "
    "LEFT JOIN measuretypes ON measures.measuretype_id=measuretypes.id) "
    "LEFT JOIN attempts ON measures.attempt_id=attempts.id "
    "WHERE events.name='" + current_event + "';")
measures = pd.read_sql(sql_measures, conn)
measures

### 6. Number of Completed Matches

In [ ]:
sql = "SELECT * FROM status"
pd.read_sql(sql, conn)

In [ ]:
# Set recent_matches to n to filter out all but the last n matches.
recent_matches= 12

select_recent_matches = sa.text(
    "with current AS (SELECT s.event, s.match, date from schedules sched, "
    "status s WHERE sched.event = s.event "
    "AND sched.match = s.match limit 1 ) "


    "SELECT * FROM ( "
    "SELECT row_number() over (partition by team order by sched.date desc) as r, "
    " sched.* from schedules sched, current c WHERE sched.event = c.event and sched.date <= c.date )"
    " row_schedule WHERE row_schedule.r <= " + str(
        recent_matches) + " ORDER by team, date desc")
df = pd.read_sql(select_recent_matches, conn)
num_matches = df[['team', 'level']].groupby('team').count()
num_matches.columns = ['matches']
df

### 7. Gears Dataframe
Verify a dataframe is visible below this window.

In [ ]:
# Create summary dataframe for placing gears
gears_df = measures.query("task == 'placeGear'").query("phase == 'auto' or phase == 'teleop'").groupby('team').sum()
gears_df['average'] = gears_df['successes'] / num_matches['matches']
gears_df = gears_df.sort_values('average', ascending = False)

gears_df.head()

### 8. Gears Chart
Verify a bar chart is visible below this cell.

In [ ]:
plt.figure()
gears_df.loc[:, "average"].plot(kind="bar", figsize=(17, 8), title="Average Gears Placed per Match \n 2017 Turing Division",
                                color="purple", fontsize=14)